<a href="https://colab.research.google.com/github/skywalker0803r/oil_web/blob/master/%E7%AC%AC%E5%9B%9B%E7%89%88%E6%A8%A1%E7%B5%84%E9%96%8B%E7%99%BC_SVR_%E5%BA%8F%E5%88%97%E9%A0%90%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
from sklearn.svm import SVR
from tqdm import tqdm_notebook as tqdm
import numpy as np
from sklearn.metrics import r2_score,mean_squared_error
from math import sqrt
from sklearn.preprocessing import StandardScaler
import warnings;warnings.simplefilter('ignore')
from sklearn.pipeline import Pipeline
import joblib
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# load raw data

In [0]:
train = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/train_4565.csv',index_col=0)
test = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/test_170.csv',index_col=0)

# data for modeling

In [3]:
cols = train.columns.tolist()
x_cols = ['T10','T50','T90','N+A']
y_cols = ['C5NP','C5IP','C5N','C6NP','C6IP','C6N','C6A','C7NP','C7IP','C7N','C7A',
'C8NP','C8IP','C8N','C8A','C9NP','C9IP','C9N','C9A','C10NP','C10IP','C10N','C10A']
X_train = train[x_cols]
X_test = test[x_cols]
y_train = train[y_cols]
y_test = test[y_cols]
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4565, 4)
(4565, 23)
(170, 4)
(170, 23)


# define model

In [0]:
class custom_model(object):
  def __init__(self,x_cols,y_cols):
    self.x_cols = x_cols
    self.y_cols = y_cols
    self.model_23 = {}
    #初始化23個model
    for y_name in y_cols:
      self.model_23[y_name] = Pipeline([('scaler',StandardScaler()),('reg',SVR(C=0.3))])
  
  def fit(self,X,y):
    for y_name in tqdm(self.y_cols):
      # fit
      self.model_23[y_name].fit(X,y[y_name])
      # 序列式將y_pred併入X
      y_pred = self.model_23[y_name].predict(X)
      X.loc[:,y_name] = y_pred
    # 還原X
    X = X[self.x_cols]
  
  def predict(self,X):
    results = pd.DataFrame(index=[*range(len(X))],columns=self.y_cols)
    for y_name in self.y_cols:
      # predict
      y_pred = self.model_23[y_name].predict(X)
      results.loc[:,y_name] = y_pred
      # 序列式將y_pred併入X
      X.loc[:,y_name] = y_pred
    # 還原X
    X = X[self.x_cols]
    # 預測結果
    return results.values

# fit model

In [5]:
model = custom_model(x_cols,y_cols)
model.fit(X_train,y_train)

# model predict

In [6]:
y_pred = model.predict(X_test)
y_pred.shape

(170, 23)

# metric

In [7]:
y_pred = pd.DataFrame(y_pred,index=[*range(len(X_test))],columns=y_cols)
y_pred.head()

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.190894,0.807816,0.160609,3.573410,2.227260,3.516232,0.654154,7.918605,6.272781,9.455625,3.257780,7.923163,8.302073,6.966316,5.818806,5.146293,9.245465,4.937084,3.466927,1.127751,5.198255,0.668779,0.687152
1,0.137756,0.116131,0.102770,3.273672,1.100118,2.163449,0.398651,10.462490,7.612349,6.337935,2.885223,9.681833,10.256803,4.981778,5.586067,6.757531,10.725721,4.448437,3.891992,1.188590,6.041288,0.597686,0.706335
2,0.129679,0.107544,0.105217,3.252565,1.090545,2.095699,0.370173,10.462874,7.574888,6.133792,2.822014,9.715474,10.298116,4.816792,5.483437,6.841115,10.733315,4.356014,3.957389,1.313952,6.230529,0.602698,0.854748
3,0.116662,0.105996,0.103472,3.401498,1.171461,2.103547,0.412287,10.469417,7.474605,6.069143,2.846285,9.805304,10.347694,4.833161,5.573599,6.879981,10.817638,4.367383,3.941348,1.197543,6.175576,0.543607,0.638214
4,0.150804,0.123865,0.102519,3.239423,1.122442,2.139366,0.405910,10.306411,7.425585,6.255281,2.871297,9.636309,10.172680,4.979009,5.660119,6.796300,10.771386,4.415727,4.008064,1.231850,6.220064,0.584412,0.733545


In [8]:
y_test.head()

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.142,0.616,0.217,3.745,2.577,4.028,0.557,7.669,5.9900,10.206,3.075,7.920,8.036,7.039,5.438,5.263,9.537,4.877,3.481,1.088,5.652,0.618,0.597
1,0.122,0.078,0.029,3.955,1.563,2.407,0.400,10.016,7.2633,6.573,2.809,9.544,9.815,5.135,5.482,6.793,10.850,4.308,3.963,1.069,6.025,0.585,0.484
2,0.098,0.064,0.020,3.920,1.447,2.296,0.396,10.236,7.3480,6.259,2.797,9.655,9.986,4.942,5.453,6.947,10.996,4.269,3.997,1.034,6.107,0.543,0.456
3,0.122,0.085,0.021,3.900,1.437,2.274,0.397,10.259,7.3560,6.234,2.820,9.670,10.146,4.928,5.476,6.953,11.026,4.264,3.982,1.013,6.054,0.545,0.449
4,0.072,0.049,0.015,3.873,1.369,2.206,0.489,10.213,7.3190,6.009,2.968,9.564,10.052,4.769,6.584,6.866,10.892,4.169,4.227,0.930,5.828,0.519,0.463


In [0]:
def mape(a, b):
    a = np.array(a)
    b = np.array(b)
    mask = a != 0
    return (np.fabs(a-b)/a)[mask].mean()

def metric(y_t,y_p,name):
  res = {'R2' : max(r2_score(y_t,y_p), 0),
         'RMSE': sqrt(mean_squared_error(y_t,y_p)),
         'MAPE': mape(y_t,y_p)}
  return pd.DataFrame(res,index=[name])

In [0]:
res = pd.DataFrame()
for y_name in y_cols:
  res = res.append(metric(y_test[y_name],y_pred[y_name],y_name))
res.loc['AVG'] = res.mean()

In [11]:
res.style.background_gradient()

,R2,RMSE,MAPE
C5NP,0.885182,0.289574,0.293926
C5IP,0.76416,0.248511,0.591246
C5N,0.795032,0.0619779,2.2044
C6NP,0.228689,0.355317,0.0655586
C6IP,0.8659,0.382771,0.097805
C6N,0.872946,0.590288,0.0545211
C6A,0.541011,0.115748,0.165315
C7NP,0.97984,0.312267,0.0331917
C7IP,0.868863,0.400153,0.0480721
C7N,0.981117,0.57266,0.0366661


# check 是否 pass

In [12]:
c1 = (len(res.loc[res.RMSE > 0.6]) == 0)
c2 = (res.loc['AVG','RMSE'] < 0.4)
print(c1,c2)

True True


# model save

In [13]:
joblib.dump(model,'model.pkl')
print('save done')

save done
